In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

2021-10-31 11:22:32.996261: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-31 11:22:32.996303: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'


# Learning rate
learning_rate = 1e-4

In [3]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [4]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [5]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [6]:
synth_delta = 0.99
synth_T_serial = 5.8
synth_rho_M = 0.76
synth_lambda_M = 4.7
synth_nu_M = 3.1

In [7]:
delta = {}
delta[Vax.total.value] = {}
delta[Vax.total.value]['prior'] = {'a': 18, 'b': 2}
# delta is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.1
# I cant figure out the math to do for the scale parameter
delta[Vax.total.value]['posterior_init'] = {'loc': np.log(synth_delta/(1-synth_delta)), 'scale':0.001}

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}
# T_serial is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 5.8
# I cant figure out the math to do for the scale parameter
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(synth_T_serial),
                                         'scale':0.001}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}
# rho is 0-1, so we will sigmoid the output of the variational posterior
# Here we use a reverse sigmoid to convert from a mean of 0.76
# I cant figure out the math to do for the scale parameter
rho_M[Vax.total.value]['posterior_init'] = {'loc': np.log(synth_rho_M/(1-synth_rho_M)),
                                      'scale':0.001}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}
# lambda is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 4.7
# I cant figure out the math to do for the scale parameter
lambda_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(synth_lambda_M),
                                         'scale':0.001}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}
# nu is positive, so we will softplus the output of the variational posterior
# Here we use a reverse softplus to convert from a mean of 3.1
# I cant figure out the math to do for the scale parameter
nu_M[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(synth_nu_M),
                                     'scale':0.001}


2021-10-31 11:22:35.135848: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:2d:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-10-31 11:22:35.136122: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-31 11:22:35.136162: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-10-31 11:22:35.136196: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-10-31 11:22:35.136227: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

In [8]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]),dtype=tf.float32),
                                                         'scale': 1})#tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]/10),dtype=tf.float32)})

In [9]:
model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                 delta, T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000)
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.SGD(
    learning_rate=learning_rate, #momentum=0.98, nesterov=False, name='SGD'
)

In [10]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)


In [11]:
warmup_A_params

{-1: {'prior': [{'loc': 2518.60923786445, 'scale': 251.86092378644497},
   {'loc': 2520.22371776634, 'scale': 252.022371776634},
   {'loc': 2518.626798999444, 'scale': 251.8626798999444}],
  'posterior_init': [{'loc': <tf.Tensor: shape=(), dtype=float32, numpy=2518.6091>,
    'scale': 1},
   {'loc': <tf.Tensor: shape=(), dtype=float32, numpy=2520.2236>, 'scale': 1},
   {'loc': <tf.Tensor: shape=(), dtype=float32, numpy=2518.6267>,
    'scale': 1}]}}

In [12]:
delta

{-1: {'prior': {'a': 18, 'b': 2},
  'posterior_init': {'loc': 4.595119850134589, 'scale': 0.001}}}

In [13]:
A_vals = []
M_vals =[]
for day in range(transition_window):
    A_vals.append(warmup_A_params[-1]['prior'][day]['loc'])

for day in range(len(x_train)):
    yesterday_asymp = A_vals[-1]
    
    today_asymp = yesterday_asymp*synth_delta*x_train[day]**(1/synth_T_serial)
    A_vals.append(today_asymp)
    
    today_M = 0
    pi_M=[]
    for j in range(transition_window):
        
        
        lambda_M_fix = synth_lambda_M
        nu_M_fix = synth_nu_M
        poisson_dist = scipy.stats.poisson(lambda_M_fix)
        pi_M_j_ago = poisson_dist.logpmf(j+1)/nu_M_fix
        
        pi_M.append(pi_M_j_ago)
    
    pi_M = scipy.special.softmax(pi_M)
        
    for j in range(transition_window):
        j_ago_asymp = A_vals[day-j-1]
        today_M += j_ago_asymp*synth_rho_M*pi_M[j]
        
    M_vals.append(today_M)

In [14]:
model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                 delta, T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000)
loss = LogPoissonProb() 

In [15]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)

with tf.GradientTape() as tape:
    result = model.call(x_train)
    loss_val = loss(y_test, result)
    loss_val += sum(model.losses)

Pre-softmax pi_M [[-0.7348403  -1.1045905  -0.8057436  ... -0.76970786 -0.89253795
  -0.9710288 ]
 [-0.5430172  -0.80578196 -0.5875981  ... -0.56242603 -0.65386254
  -0.7105152 ]
 [-0.47123826 -0.6409987  -0.478045   ... -0.4437362  -0.54862607
  -0.55834085]]
Delta 0.9874223470687866
T serial 5.798177719116211
rho_M 0.7447925806045532
lambda M 4.7024054527282715
pi_M [0.25895503 0.3428026  0.39824238]


2021-10-31 11:22:37.664888: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


In [16]:
grads = tape.gradient(loss_val, model.trainable_weights)
grads

2021-10-31 11:22:37.879518: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


[<tf.Tensor: shape=(), dtype=float32, numpy=-35374.227>,
 <tf.Tensor: shape=(), dtype=float32, numpy=8425.702>,
 <tf.Tensor: shape=(), dtype=float32, numpy=52.744286>,
 <tf.Tensor: shape=(), dtype=float32, numpy=165.65366>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-9939.47>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3310.6409>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0041489713>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.31114295>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.76813644>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.6323154>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0036274716>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0011369425>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0061025675>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.001506672>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-13.615356>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.45699355>]

In [17]:
loss_val

<tf.Tensor: shape=(), dtype=float32, numpy=32099.527>

In [18]:
loss_val

<tf.Tensor: shape=(), dtype=float32, numpy=32099.527>

In [25]:
with tf.GradientTape() as tape:
    result = model.call(x_train)
    loss_val = loss(tf.cast(np.array(M_vals),dtype=tf.float32), result)
    loss_val += sum(model.losses)

Pre-softmax pi_M [[-1.9353518  -0.7593234  -1.0184203  ... -1.1489791  -1.4848338
  -0.6912029 ]
 [-1.4419014  -0.56447643 -0.74245167 ... -0.84091634 -1.0854495
  -0.5139835 ]
 [-1.1187007  -0.499051   -0.6004087  ... -0.6604573  -0.8551549
  -0.4547904 ]]
Delta 0.9996104836463928
T serial 5.372091293334961
rho_M 0.6834198236465454
lambda M 4.71892786026001
pi_M [0.25777388 0.34284753 0.3993786 ]


In [26]:
grads = tape.gradient(loss_val, model.trainable_weights)
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=3771.659>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-251.16531>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-3215.6064>,
 <tf.Tensor: shape=(), dtype=float32, numpy=534.15375>,
 <tf.Tensor: shape=(), dtype=float32, numpy=29631.934>,
 <tf.Tensor: shape=(), dtype=float32, numpy=164.31656>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.012386041>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.33945695>,
 <tf.Tensor: shape=(), dtype=float32, numpy=110.46721>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-21.682081>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.027709698>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.0007829711>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.048783172>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.00057341997>,
 <tf.Tensor: shape=(), dtype=float32, numpy=37.83845>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.08075323>]

In [27]:
model.trainable_weights

[<tf.Variable 'delta_A_loc_-1:0' shape=() dtype=float32, numpy=7.8859735>,
 <tf.Variable 'delta_A_scale_-1:0' shape=() dtype=float32, numpy=-0.9774477>,
 <tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=5.3628316>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=-0.54573506>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=0.7737106>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=-1.7369808>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=4.690876>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=0.00046391948>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=3.0305226>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=0.0051967604>,
 <tf.Variable 'warmup_A_loc_0_-1:0' shape=() dtype=float32, numpy=2518.6091>,
 <tf.Variable 'warmup_A_scale_0_-1:0' shape=() dtype=float32, numpy=0.9999997>,
 <tf.Variable 'warmup_A_loc_1_-1:0' shape=() dtype=float32, numpy=2520.223

In [22]:
loss_val

<tf.Tensor: shape=(), dtype=float32, numpy=11667.086>

In [23]:
loss_val

<tf.Tensor: shape=(), dtype=float32, numpy=11667.086>

In [24]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([y_test]),
         epochs=2000, batch_size=0,
        callbacks=[])

Epoch 1/2000
Pre-softmax pi_M [[-1.4147328  -0.8765375  -0.92787045 ... -0.99218035 -0.89137816
  -1.294756  ]
 [-1.037609   -0.6400245  -0.68963    ... -0.7233113  -0.6514466
  -0.94671994]
 [-0.8117139  -0.52688766 -0.609328   ... -0.5846623  -0.5395572
  -0.74534285]]
Delta 0.9869816899299622
T serial 5.786071300506592
rho_M 0.7408556938171387
lambda M 4.709111213684082
pi_M [0.25860357 0.34285265 0.39854378]
1/1 [==============================] - 2s 2s/step - loss: 34225.3125
Epoch 2/2000
Pre-softmax pi_M [[-0.75526285 -0.79250354 -0.7762477  ... -0.84379715 -0.7270942
  -1.1893463 ]
 [-0.5517179  -0.59105647 -0.56800586 ... -0.6162308  -0.53007
  -0.86860645]
 [-0.45561454 -0.5273889  -0.4737844  ... -0.5079715  -0.4249247
  -0.6866433 ]]
Delta 0.9997457265853882
T serial 5.82442045211792
rho_M 0.8865716457366943
lambda M 4.669232368469238
pi_M [0.25880268 0.3430207  0.39817664]
1/1 [==============================] - 2s 2s/step - loss: 10206.5703
Epoch 3/2000
Pre-softmax pi_M [[-0

KeyboardInterrupt: 

In [ ]:
model.trainable_weights

In [ ]:
r_t

In [ ]:
x_train

In [ ]:
len(y_test)

In [ ]:
A_vals

In [ ]:
len(M_vals)

In [ ]:
model.trainable_weights

In [ ]:
np.log(-2.9/(1-2.9))

In [ ]:
x_train

In [ ]:
1.1**(1/5.8)

In [ ]:
df.loc[train_start:train_end, 
       'asymp'].values

In [29]:
y_test

<tf.Tensor: shape=(123,), dtype=float32, numpy=
array([1883.1843 , 1879.9614 , 1878.4843 , 1874.889  , 1867.625  ,
       1855.1506 , 1839.0627 , 1819.4205 , 1797.5369 , 1772.8456 ,
       1745.3354 , 1713.0474 , 1676.8483 , 1637.9723 , 1595.6298 ,
       1550.9956 , 1505.9484 , 1460.1703 , 1411.5771 , 1360.6824 ,
       1309.5687 , 1257.2211 , 1206.9117 , 1158.9786 , 1112.176  ,
       1065.396  , 1019.1018 ,  974.7162 ,  931.76184,  891.36066,
        854.1387 ,  819.2815 ,  786.52826,  754.50806,  724.3344 ,
        696.807  ,  669.4488 ,  646.7868 ,  625.5422 ,  605.7364 ,
        587.4074 ,  570.8427 ,  555.80206,  542.41534,  530.70746,
        520.6189 ,  512.4302 ,  504.1964 ,  498.84677,  495.26813,
        493.92334,  494.2876 ,  496.60837,  500.79382,  506.81006,
        514.7886 ,  524.72736,  536.65625,  550.4345 ,  566.6077 ,
        584.84607,  605.1895 ,  627.5542 ,  651.947  ,  678.33746,
        706.6682 ,  736.8512 ,  768.7289 ,  801.9794 ,  836.5681 ,
        872.29

In [28]:
tf.reduce_mean(result, axis=1)

<tf.Tensor: shape=(123,), dtype=float32, numpy=
array([1721.6371 , 1720.6525 , 1717.0166 , 1710.3861 , 1700.644  ,
       1687.5132 , 1670.8053 , 1650.4231 , 1626.364  , 1598.7163 ,
       1567.6592 , 1533.4565 , 1496.4475 , 1457.034  , 1415.6584 ,
       1372.7823 , 1328.8628 , 1284.3438 , 1239.6488 , 1195.1702 ,
       1151.2543 , 1108.1935 , 1066.2264 , 1025.5432 ,  986.2899 ,
        948.574  ,  912.4698 ,  878.02234,  845.2526 ,  814.16327,
        784.746  ,  756.9894 ,  730.87994,  706.40436,  683.54926,
        662.30334,  642.6584 ,  624.6101 ,  608.15924,  593.31036,
        580.07166,  568.4524 ,  558.4641 ,  550.1219 ,  543.4456 ,
        538.45856,  535.1858 ,  533.65424,  533.8938 ,  535.9386 ,
        539.8261 ,  545.5945 ,  553.2818 ,  562.9245 ,  574.55634,
        588.2066 ,  603.8985 ,  621.645  ,  641.445  ,  663.2785 ,
        687.1046 ,  712.86206,  740.4679 ,  769.8131 ,  800.7586 ,
        833.1359 ,  866.7534 ,  901.403  ,  936.86273,  972.8991 ,
       1009.27

In [ ]:
result

In [ ]:
x_train

In [ ]:
y_test

In [ ]:
tf.cast(df.loc[train_start:test_end,'asymp'], dtype=tf.float32)